In [1]:
import json
from pprint import pprint as pp
import pandas as pd
import sys
import os
from IPython.core.display import HTML, display
from mapboxgl.errors import TokenError
from mapboxgl import templates
from mapboxgl.viz import CircleViz, HexbinViz
from mapboxgl.utils import df_to_geojson, create_radius_stops, scale_between , hexbin , df_to_hexbin
from mapboxgl.colors import create_color_stops
import numpy as np
import pandas as pd
import sys
import os

token = 'pk.eyJ1Ijoic2F0dmlybGlkaGFyIiwiYSI6ImNqOHh4a3Q3ejBwMDcyd25yZWxycm9iczgifQ.3Gnf8TiSwPFrrlf-hqRZbA'

sys.path.append("..")

def jenks_matrices_init(data, n_classes):
    #fill the matrices with data+1 arrays of n_classes 0s
    lower_class_limits = []
    variance_combinations = []
    for i in range(0, len(data)+1):
        temp1 = []
        temp2 = []
        for j in range(0, n_classes+1):
            temp1.append(0.)
            temp2.append(0.)
        lower_class_limits.append(temp1)
        variance_combinations.append(temp2)

    inf = float('inf')
    for i in range(1, n_classes+1):
        lower_class_limits[1][i] = 1.
        variance_combinations[1][i] = 0.
        for j in range(2, len(data)+1):
            variance_combinations[j][i] = inf

    return lower_class_limits, variance_combinations

def jenks_matrices(data, n_classes):
    lower_class_limits, variance_combinations = jenks_matrices_init(data, n_classes)

    variance = 0.0
    for l in range(2, len(data)+1):
        sum = 0.0
        sum_squares = 0.0
        w = 0.0
        for m in range(1, l+1):
            # `III` originally
            lower_class_limit = l - m + 1
            val = data[lower_class_limit-1]

            # here we're estimating variance for each potential classing
            # of the data, for each potential number of classes. `w`
            # is the number of data points considered so far.
            w += 1

            # increase the current sum and sum-of-squares
            sum += val
            sum_squares += val * val

            # the variance at this point in the sequence is the difference
            # between the sum of squares and the total x 2, over the number
            # of samples.
            variance = sum_squares - (sum * sum) / w

            i4 = lower_class_limit - 1

            if i4 != 0:
                for j in range(2, n_classes+1):
                    if variance_combinations[l][j] >= (variance + variance_combinations[i4][j - 1]):
                        lower_class_limits[l][j] = lower_class_limit
                        variance_combinations[l][j] = variance + variance_combinations[i4][j - 1]

        lower_class_limits[l][1] = 1.
        variance_combinations[l][1] = variance

    return lower_class_limits, variance_combinations

def get_jenks_breaks(data, lower_class_limits, n_classes):
    k = len(data) - 1
    kclass = [0.] * (n_classes+1)
    countNum = n_classes

    kclass[n_classes] = data[len(data) - 1]
    kclass[0] = data[0]

    while countNum > 1:
        elt = int(lower_class_limits[k][countNum] - 2)
        kclass[countNum - 1] = data[elt]
        k = int(lower_class_limits[k][countNum] - 1)
        countNum -= 1

    return kclass

def jenks(data, n_classes):
    if n_classes > len(data): return

    data.sort()

    lower_class_limits, _ = jenks_matrices(data, n_classes)

    return get_jenks_breaks(data, lower_class_limits, n_classes)

In [3]:
 Tree Locations
df = pd.read_csv('ny_trees_50000.csv')
df['lat'] = df['latitude']
df['lon'] = df['longitude']
data = df_to_geojson(df, properties=['tree_id'])

viz = CircleViz(data,  access_token=token, height='400px')
viz.center = (-73.7, 40.7)
viz.zoom = 8.5
viz.show()
# Final Result for Part 1



In [4]:
Filtered Trees
df = pd.read_csv('2015_Street_Tree_Census_-_Tree_Data.csv')
df = df[df.status == "Stump"]
df['lat'] = df['latitude']
df['lon'] = df['longitude']
data = df_to_geojson(df, properties=['address'])
viz = CircleViz(data,  access_token=token, height='400px')
viz.center = (-73.7, 40.7)
viz.zoom = 8.5
viz.show() 
# Final Result for Part 2

In [6]:
Trees by Type
df = pd.read_csv('ny_trees_50000.csv')
df2 = df.spc_common.value_counts().head(8)
df = df[df["spc_common"].isin(list(df2.index))]
df['lat'] = df['latitude']
df['lon'] = df['longitude']
data = df_to_geojson(df, properties=['spc_common'])
viz = CircleViz(data,  access_token=token, height='400px')
color_stops = create_color_stops(df2.index.tolist(), colors='YlGnBu')
viz.color_property = "spc_common"
viz.color_type = "categorical"
viz.color_stops = color_stops
viz.center = (-73.7, 40.7)
viz.zoom = 8.5
viz.show()
# Final Result for Part 3

In [7]:
Aggregation by Location 
df = pd.read_csv('2015_Street_Tree_Census_-_Tree_Data.csv')
df['lat'] = df['latitude']
df['lon'] = df['longitude']
data = df_to_hexbin(df, radius=250)
viz = HexbinViz(data[0],access_token=token)
x = int(data[1]/8)
color_stops = create_color_stops(list(range(1, data[1], x)), colors='YlGnBu')
viz.color_stops = color_stops
viz.color_property = "count"
viz.color_type = "interval"
viz.center = (-73.2, 40.7)
viz.zoom = 8
viz.show() 
# Final Result for Part 4